In [2]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.8 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import warnings
import joblib
import plotly.express as px
from google.colab import drive
from datetime import date, timedelta
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# from catboost import CatBoostClassifier, CatBoostRegressor, Pool
# from plot_metric.functions import BinaryClassification
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import log_loss, roc_auc_score
import seaborn as sns
import plotly.express as px

warnings.filterwarnings('ignore')
pd.options.display.max_columns = 500

from google.colab import auth
import gspread
from google.auth import default
from google.cloud import bigquery
client = bigquery.Client(project='dev-sd-lake')

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [10]:
sales_prob_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/sales_prob_price_model.pkl")
domestic_export_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/domestic_export_price_model.pkl")
time_at_a_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/time_at_a_price_model.pkl")
amount_at_a_price_model = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/amount_at_a_price_model.pkl")
X_val_sales_prob = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/X_val_sales_prob.pkl")
X_val_domestic_export = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/X_val_domestic_export.pkl")

# default 1,2,3,4 for fern

In [5]:
test_data = X_val_sales_prob.sample(1)
test_data = test_data[['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'total_units', 'shelf_life_remaining_days',
              'time', 'listing_condition', 'retail_price_per_unit_usd', 'order_price_per_unit_usd']]
test_data = test_data.rename(columns = {'order_price_per_unit_usd' : 'asking_price_per_unit_usd'})

In [6]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd
399997,E2S06220,puma,fashion_-_footwear,women's_footwear,excess2sell,2.0,NaN,NaN,excess,11.498159,3.490883


In [7]:
def get_dummy_prices(x):
  return [x*0.7, x*0.8, x*0.9, x, x*1.1, x*1.2, x*1.3]

test_data['new_price_per_unit_usd'] = test_data['asking_price_per_unit_usd'].apply(get_dummy_prices)
test_data = test_data.explode('new_price_per_unit_usd').reset_index(drop = True)
test_data['order_price_per_unit_usd'] = test_data['new_price_per_unit_usd']
test_data = test_data.drop('new_price_per_unit_usd', axis = 1)

In [8]:
test_data['probs'] = sales_prob_price_model.predict_proba(test_data)[:,1]
test_data['probs'] = test_data['probs'] * 100
test_data['discount / price_hike_from_asking'] = (test_data['order_price_per_unit_usd'] - test_data['asking_price_per_unit_usd']) / test_data['asking_price_per_unit_usd'] * 100

In [9]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd,order_price_per_unit_usd,probs,discount / price_hike_from_asking
0,E2S06220,puma,fashion_-_footwear,women's_footwear,excess2sell,2.0,NaN,NaN,excess,11.498159,3.490883,2.443618,64.196152,-30.0
1,E2S06220,puma,fashion_-_footwear,women's_footwear,excess2sell,2.0,NaN,NaN,excess,11.498159,3.490883,2.792706,61.166192,-20.0
2,E2S06220,puma,fashion_-_footwear,women's_footwear,excess2sell,2.0,NaN,NaN,excess,11.498159,3.490883,3.141794,50.634083,-10.0
3,E2S06220,puma,fashion_-_footwear,women's_footwear,excess2sell,2.0,NaN,NaN,excess,11.498159,3.490883,3.490883,77.485437,0.0
4,E2S06220,puma,fashion_-_footwear,women's_footwear,excess2sell,2.0,NaN,NaN,excess,11.498159,3.490883,3.839971,43.287834,10.0
5,E2S06220,puma,fashion_-_footwear,women's_footwear,excess2sell,2.0,NaN,NaN,excess,11.498159,3.490883,4.189059,29.392531,20.0
6,E2S06220,puma,fashion_-_footwear,women's_footwear,excess2sell,2.0,NaN,NaN,excess,11.498159,3.490883,4.538147,35.718448,30.0


# default 5,6

In [ ]:
test_data = X_val_domestic_export.sample(1)
test_data = test_data[['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'total_units', 'shelf_life_remaining_days',
              'time', 'listing_condition', 'retail_price_per_unit_usd', 'order_price_per_unit_usd', 'domestic_export']]
test_data = test_data.rename(columns = {'order_price_per_unit_usd' : 'asking_price_per_unit_usd'})

In [ ]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd,domestic_export
421500,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic


In [ ]:
def get_dummy_prices(x):
  return [x*0.7, x*0.8, x*0.9, x, x*1.1, x*1.2, x*1.3]

test_data['new_price_per_unit_usd'] = test_data['asking_price_per_unit_usd'].apply(get_dummy_prices)
test_data = test_data.explode('new_price_per_unit_usd').reset_index(drop = True)
test_data['order_price_per_unit_usd'] = test_data['new_price_per_unit_usd']
test_data = test_data.drop('new_price_per_unit_usd', axis = 1)

In [ ]:
test_data['probs'] = domestic_export_price_model.predict_proba(test_data)[:,1]
test_data['probs'] = test_data['probs'] * 100
test_data['discount / price_hike_from_asking'] = (test_data['order_price_per_unit_usd'] - test_data['asking_price_per_unit_usd']) / test_data['asking_price_per_unit_usd'] * 100

In [ ]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,time,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd,domestic_export,order_price_per_unit_usd,probs,discount / price_hike_from_asking
0,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,18.341233,36.546140,-30.0
1,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,20.961409,96.005462,-20.0
2,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,23.581585,90.886952,-10.0
3,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,26.201761,96.895372,0.0
4,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,28.821938,21.368473,10.0
5,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,31.442114,7.661917,20.0
6,E2S03373,puma,fashion_-_footwear,unisex_footwear,excess2sell,1.0,NaN,NaN,excess,51.782001,26.201761,domestic,34.06229,3.916800,30.0


# default 7,8

In [22]:
X_val_time_at_a_price = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/X_val_time_at_a_price.pkl")

In [23]:
test_data = X_val_time_at_a_price.sample(1)
test_data = test_data[['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'total_units', 'shelf_life_remaining_days',
              'listing_condition', 'retail_price_per_unit_usd', 'order_price_per_unit_usd', 'domestic_export', 'sellability']]
test_data = test_data.rename(columns = {'order_price_per_unit_usd' : 'asking_price_per_unit_usd'})

In [24]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd,domestic_export,sellability
192756,E2S06943,puma,fashion_-_footwear,unisex_footwear,excess2sell,2.233885,NaN,excess,46.027166,20.8514,domestic,0


In [25]:
# Generate 7 price variations around the original asking price
# Includes: 30%, 20%, 10% discount, original price, and 10%, 20%, 30% markup
def get_dummy_prices(x):
    return [x*0.7, x*0.8, x*0.9, x, x*1.1, x*1.2, x*1.3]

# Apply simulated price variations to each row
test_data['new_price_per_unit_usd'] = test_data['asking_price_per_unit_usd'].apply(get_dummy_prices)

# Expand each row into 7 rows, one per simulated price
test_data = test_data.explode('new_price_per_unit_usd').reset_index(drop=True)

# Assign each simulated price to a new column used for prediction
test_data['order_price_per_unit_usd'] = test_data['new_price_per_unit_usd']

# Drop the temporary list column after expansion
test_data = test_data.drop('new_price_per_unit_usd', axis=1)

# Predict estimated time to sale at each price point using a trained model
test_data['time'] = time_at_a_price_model.predict(test_data)

# Convert log-scaled predictions back to original scale (assumes model was trained on log1p)
test_data['time'] = np.expm1(test_data['time'])

# Compute percentage difference from original asking price to simulate discount/markup effect
test_data['discount / price_hike_from_asking'] = (
    (test_data['order_price_per_unit_usd'] - test_data['asking_price_per_unit_usd']) /
    test_data['asking_price_per_unit_usd'] * 100
)

In [28]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,total_units,shelf_life_remaining_days,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd,domestic_export,sellability,order_price_per_unit_usd,time,discount / price_hike_from_asking
0,E2S06943,puma,fashion_-_footwear,unisex_footwear,excess2sell,2.233885,NaN,excess,46.027166,20.8514,domestic,0,14.59598,55.086288,-30.0
1,E2S06943,puma,fashion_-_footwear,unisex_footwear,excess2sell,2.233885,NaN,excess,46.027166,20.8514,domestic,0,16.68112,60.545645,-20.0
2,E2S06943,puma,fashion_-_footwear,unisex_footwear,excess2sell,2.233885,NaN,excess,46.027166,20.8514,domestic,0,18.76626,62.941815,-10.0
3,E2S06943,puma,fashion_-_footwear,unisex_footwear,excess2sell,2.233885,NaN,excess,46.027166,20.8514,domestic,0,20.8514,65.553193,0.0
4,E2S06943,puma,fashion_-_footwear,unisex_footwear,excess2sell,2.233885,NaN,excess,46.027166,20.8514,domestic,0,22.93654,68.584734,10.0
5,E2S06943,puma,fashion_-_footwear,unisex_footwear,excess2sell,2.233885,NaN,excess,46.027166,20.8514,domestic,0,25.02168,69.795322,20.0
6,E2S06943,puma,fashion_-_footwear,unisex_footwear,excess2sell,2.233885,NaN,excess,46.027166,20.8514,domestic,0,27.10682,85.565647,30.0


# default 9, 10

In [69]:
X_val_amount_at_a_price = joblib.load("/content/drive/MyDrive/PSI_Modeling_Data/Fern/X_val_amount_at_a_price.pkl")

In [70]:
test_data = X_val_amount_at_a_price.sample(1)
test_data = test_data[['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'shelf_life_remaining_days',
              'listing_condition', 'retail_price_per_unit_usd', 'order_price_per_unit_usd', 'domestic_export', 'sellability', 'time']]
test_data = test_data.rename(columns = {'order_price_per_unit_usd' : 'asking_price_per_unit_usd'})

In [71]:
# Function to generate 7 price points around the original price (from 30% discount to 30% hike)
def get_dummy_prices(x):
    return [x*0.7, x*0.8, x*0.9, x, x*1.1, x*1.2, x*1.3]

# Apply price simulation to each row in 'asking_price_per_unit_usd'
test_data['new_price_per_unit_usd'] = test_data['asking_price_per_unit_usd'].apply(get_dummy_prices)

# Expand each row into 7 rows, one for each simulated price
test_data = test_data.explode('new_price_per_unit_usd').reset_index(drop=True)

# Assign simulated prices to the column used for prediction
test_data['order_price_per_unit_usd'] = test_data['new_price_per_unit_usd']

# Drop the temporary column as it's no longer needed
test_data = test_data.drop('new_price_per_unit_usd', axis=1)

# Predict total units sold using the trained model at each price point
test_data['total_units'] = amount_at_a_price_model.predict(test_data)

# Convert log-transformed model outputs back to original scale
test_data['total_units'] = np.expm1(test_data['total_units'])

# Calculate discount or markup percentage relative to original asking price
test_data['discount / price_hike_from_asking'] = (
    (test_data['order_price_per_unit_usd'] - test_data['asking_price_per_unit_usd']) /
    test_data['asking_price_per_unit_usd'] * 100
)

In [72]:
test_data

,sku_number,brand,product_category,product_subcategory,seller_name,shelf_life_remaining_days,listing_condition,retail_price_per_unit_usd,asking_price_per_unit_usd,domestic_export,sellability,time,order_price_per_unit_usd,total_units,discount / price_hike_from_asking
0,68733292,tresemme,hair_care,conditioner,ulph,808.0,fresh,11.94,6.498771,domestic,0,65.162456,4.549139,9.687490,-30.0
1,68733292,tresemme,hair_care,conditioner,ulph,808.0,fresh,11.94,6.498771,domestic,0,65.162456,5.199017,5.705129,-20.0
2,68733292,tresemme,hair_care,conditioner,ulph,808.0,fresh,11.94,6.498771,domestic,0,65.162456,5.848894,6.316343,-10.0
3,68733292,tresemme,hair_care,conditioner,ulph,808.0,fresh,11.94,6.498771,domestic,0,65.162456,6.498771,8.920698,0.0
4,68733292,tresemme,hair_care,conditioner,ulph,808.0,fresh,11.94,6.498771,domestic,0,65.162456,7.148648,7.369663,10.0
5,68733292,tresemme,hair_care,conditioner,ulph,808.0,fresh,11.94,6.498771,domestic,0,65.162456,7.798525,7.395852,20.0
6,68733292,tresemme,hair_care,conditioner,ulph,808.0,fresh,11.94,6.498771,domestic,0,65.162456,8.448402,7.507518,30.0


# high level analysis after fern inference

In [276]:
test_data = X_val_sales_prob
test_data = test_data.sample(1000)

test_data = test_data[['sku_number', 'brand', 'product_category', 'product_subcategory', 'seller_name', 'total_units', 'shelf_life_remaining_days',
              'time', 'listing_condition', 'retail_price_per_unit_usd', 'order_price_per_unit_usd']]
test_data = test_data.rename(columns = {'order_price_per_unit_usd' : 'asking_price_per_unit_usd'})
test_data = test_data[test_data['asking_price_per_unit_usd'] != 0]
test_data = test_data.reset_index(drop = True)
test_data['id'] = test_data.index

In [277]:
def get_dummy_prices(x):
  return [x*0.7, x*0.8, x*0.9, x, x*1.1, x*1.2, x*1.3]

test_data['new_price_per_unit_usd'] = test_data['asking_price_per_unit_usd'].apply(get_dummy_prices)
test_data = test_data.explode('new_price_per_unit_usd').reset_index(drop = True)
test_data['order_price_per_unit_usd'] = test_data['new_price_per_unit_usd']
test_data = test_data.drop('new_price_per_unit_usd', axis = 1)

In [278]:
test_data['probs'] = sales_prob_price_model.predict_proba(test_data)[:,1]
test_data['expected_revenue'] = test_data['probs'] * test_data['order_price_per_unit_usd']
test_data['expected_total_units_could_sell'] = test_data['probs'] * test_data['total_units']
test_data['probs'] = test_data['probs'] * 100
test_data['discount / price_hike_from_asking'] = (test_data['order_price_per_unit_usd'] - test_data['asking_price_per_unit_usd']) / test_data['asking_price_per_unit_usd'] * 100
test_data['discount / price_hike_from_asking'] = (test_data['discount / price_hike_from_asking'].astype(float).round(-1))

In [279]:
# Step 1: Find the row with max 'probs' for each SKU
best_price_df = (
    test_data.loc[test_data.groupby("sku_number")["probs"].idxmax()]
    .reset_index(drop=True)
)

# Step 2: Select the relevant columns
best_price_df = best_price_df[
    ["sku_number", "brand", "product_category", "product_subcategory", "seller_name",
     "retail_price_per_unit_usd", "order_price_per_unit_usd",
     "discount / price_hike_from_asking", "probs",
     "expected_revenue", "expected_total_units_could_sell"]
]

# Step 3: Rename columns for clarity
best_price_df = best_price_df.rename(columns={
    "discount / price_hike_from_asking": "best_discount_level",
    "probs": "max_probability",
    "expected_revenue": "estimated_revenue_at_best_price",
    "expected_total_units_could_sell": "estimated_units_could_sell_at_best_price"
})

In [280]:
best_price_df

,sku_number,brand,product_category,product_subcategory,seller_name,retail_price_per_unit_usd,order_price_per_unit_usd,best_discount_level,max_probability,estimated_revenue_at_best_price,estimated_units_could_sell_at_best_price
0,10001284,thomas_sabo_charm_club,fashion_-_accessories,jewelry,thomas_sabo,63.922728,30.067051,-30.0,29.658787,8.917523,53.089229
1,10028902,thomas_sabo,fashion_-_accessories,accessories_-_other,thomas_sabo,194.859975,42.089755,-10.0,95.680392,40.271642,373.153531
2,10032073,thomas_sabo_sterling_silver,fashion_-_accessories,jewelry,thomas_sabo,194.859975,40.920595,-30.0,96.999341,39.692707,65.959552
3,10033565,thomas_sabo_sterling_silver,fashion_-_accessories,jewelry,thomas_sabo,106.683115,30.724737,-20.0,95.858621,29.452309,233.895035
4,10033831,thomas_sabo_charm_club,fashion_-_accessories,jewelry,thomas_sabo,53.088367,28.667718,-10.0,42.133176,12.07862,17.274602
...,...,...,...,...,...,...,...,...,...,...,...
973,ZTH02630,l'oreal_paris,hair_care,hair_color,loth,10.890000,2.4492,30.0,67.646823,1.656806,243.528564
974,ZTH02666,l'oreal_paris,hair_care,hair_care_-_others,loth,3.460000,0.704,-20.0,90.215715,0.635119,490.773487
975,ZTH02779,l'oreal_paris,makeup,face_makeup,loth,11.410000,2.431478,-30.0,89.092214,2.166257,159.475063
976,ZTH02787,l'oreal_paris,hair_care,shampoo,loth,23.110000,5.664,0.0,78.427700,4.442145,21.175479


# Count of recommended discounts % or hike % from asking price USD for the entire inventory

In [281]:
best_price_df['best_discount_level'].value_counts().reset_index()

,best_discount_level,count
0,-30.0,526
1,-20.0,146
2,0.0,121
3,30.0,104
4,-10.0,31
5,20.0,31
6,10.0,19


## Getting total inventory estimated revenue by summing individual SKU's estimated revenue at the best possible price point (discount / hike).

## Calculating estimated best possible recovery peercent possible at the overall inventory level

In [282]:
revenue_possible_in_usd_at_best_possible_price_points = best_price_df['estimated_revenue_at_best_price'].sum()
total_inv_original_retail_usd_sum = best_price_df['retail_price_per_unit_usd'].sum()
print(revenue_possible_in_usd_at_best_possible_price_points, total_inv_original_retail_usd_sum)

print('Recovery percent = ', revenue_possible_in_usd_at_best_possible_price_points / total_inv_original_retail_usd_sum * 100)

3563.0260542758147 21604.84500006691
Recovery percent =  16.491791791446687


# brand wise estimated revenue and total units that can sell at best price

In [286]:
best_price_df.groupby('brand').agg({'estimated_revenue_at_best_price' : 'sum', 'estimated_units_could_sell_at_best_price' : 'sum'}).\
                sort_values(by = 'estimated_revenue_at_best_price', ascending = False).reset_index()

,brand,estimated_revenue_at_best_price,estimated_units_could_sell_at_best_price
0,puma,1077.935798,158.337838
1,home,703.423332,724.223297
2,other,251.346836,206.941708
3,l'oreal_paris,201.895277,154855.171125
4,maybelline,177.668622,54229.819884
...,...,...,...
90,timotei,0.047495,491.952596
91,malin+goetz,0.040507,73.161219
92,mama,0.023609,6.886016
93,otso,0.01828,56.999580


# product category wise estimated revenue and total units that can sell at best price

In [287]:
best_price_df.groupby('product_category').agg({'estimated_revenue_at_best_price' : 'sum', 'estimated_units_could_sell_at_best_price' : 'sum'}).\
                sort_values(by = 'estimated_revenue_at_best_price', ascending = False).reset_index()

,product_category,estimated_revenue_at_best_price,estimated_units_could_sell_at_best_price
0,"apparels,_footwear,_accessories",560.504011,7.750020e+01
1,fashion_-_footwear,527.941796,1.294946e+02
2,makeup,353.809046,1.023188e+05
3,home_improvement_and_tools,314.749307,2.405587e+02
4,other,261.50183,2.127064e+02
5,home_-_other,211.315386,2.349405e+02
6,skin_care,208.854323,3.700118e+05
7,fashion_-_apparel,192.755819,5.774845e+02
8,fashion_-_accessories,188.383323,1.545699e+03
9,hair_care,136.121315,4.469261e+06


# product subcategory wise estimated revenue and total units that can sell at best price

In [289]:
best_price_df.groupby('product_subcategory').agg({'estimated_revenue_at_best_price' : 'sum', 'estimated_units_could_sell_at_best_price' : 'sum'}).\
                sort_values(by = 'estimated_revenue_at_best_price', ascending = False).reset_index()

,product_subcategory,estimated_revenue_at_best_price,estimated_units_could_sell_at_best_price
0,footwear,560.504011,77.500195
1,kids_tops,450.950438,629.480746
2,unisex_footwear,424.448074,58.835670
3,women's_tops,302.236441,193.754175
4,women's_dresses_&_jumpsuits,159.184538,39.680501
...,...,...,...
67,home_decor,0.290699,845.399693
68,footwear_-_other,0.168503,0.008988
69,men's_outerwear,0.13671,1.043677
70,health_&_supplments_-_other,0.01828,56.999580
